In [5]:
import gymnasium as gym
from gymnasium.spaces import Discrete
from gymnasium.spaces import Box
import numpy as np
import pandas as pd

In [6]:
class TradingEnv(gym.Env):

    def __init__(self, data, training, episode_length = 250, budget=10000):
        self.portfolio_value = budget
        self.cur_row_num = 0
        self.starting_row_num = 0
        self.asset_allocation = 0.0
        self.data = data
        self.episode_length = episode_length
        self.training = training
    
        # action space: Sell 25%, sell 10%, no change, buy 10%, buy 25% (percentages are of total portfolio value, asset + cash, at each timestep)
        self.action_space = Discrete(5)

        # observation space: Close, Volume, SMA Ratio, RSI, Bandwidth, Asset Allocation
        self.observation_space = Box(low=np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
            high=np.array([np.inf, np.inf, np.inf, 100.0, np.inf, 1.0]))

    # returns the current row in dataframe with current asset allocation appended
    def _get_obs(self):
        obs = np.array(self.data.iloc[self.row_num, :])
        obs = np.append(obs, self.asset_allocation)
        return np.array(self.data.iloc[self.row_num, :])

    # returns current portfolio value
    def _get_info(self):
        return {'Portfolio Value': self.portfolio_value}

    # sets the starting row and starting asset allocation
    def reset(self, seed):
        super().reset(seed=seed)
        if self.training:
            self.starting_row_num = np.random.randint(0, len(self.data) - self.episode_length - 1)    # prevent out of bounds, also subtract 1 to avoid
                                                                                                 # weird edge cases for now (should change later)
        else:
            self.starting_row_num = 0

        self.cur_row_num = self.starting_row_num

        rand = np.random.rand()
        if rand < 0.7:
            self.asset_allocation = 0.0
        else:
            self.asset_allocation = np.random.rand()
        
        return self._get_obs(), self._get_info()

    # moves to the next row in data, updates reward and current portfolio value
    def step(self, action):
        self.cur_row_num += 1
        if (self.cur_row_num - self.starting_row_num) > self.episode_length:
            terminated = True
        truncated = False
        self.asset_allocation = self._action_to_allocation(action)
        obs = self._get_obs()
        rew = self._get_rew()
        info = self._get_info()
        return obs, rew, terminated, truncated, info
    
    # converts action to asset allocation value
    def _action_to_allocation(self, action):
        allocation_change = 0.0
        if action == 0: allocation_change = -.25
        elif action == 1: allocation_change = -.1
        elif action == 2: allocation_change = 0.0
        elif action == 3: allocation_change = .1
        else: allocation_change = 0.25
        return min(1.0, self.asset_allocation + allocation_change)
    
    # returns reward in the form of regular percent return of the total portfolio (stock + cash) over this timestep
    # need to also account for slippage and commission costs - add later
    def _get_reward(self):
        asset_change = (self.data.iloc[self.cur_row_num, 0] - self.data.iloc[self.cur_row_num - 1, 0]) / self.data.iloc[self.cur_row_num - 1, 0]
        new_portfolio_value = self.portfolio_value * (self.asset_allocation * (1.0 + asset_change) + (1.0 - self.asset_allocation))
        reward = (new_portfolio_value - self.portfolio_value) / self.portfolio_value
        self.portfolio_value = new_portfolio_value
        return reward


In [7]:
from stable_baselines3.common.env_checker import check_env

In [8]:
data = pd.read_csv('Amazon Data.csv')
data

,Date,Close,Volume,SMA Ratio,RSI,Bandwidth
0,1997-10-08 00:00:00-04:00,0.200260,56136000,1.368536,62.081040,0.451564
1,1997-10-09 00:00:00-04:00,0.187500,79008000,1.345808,53.301818,0.412582
2,1997-10-10 00:00:00-04:00,0.195052,75744000,1.323756,42.939429,0.407098
3,1997-10-13 00:00:00-04:00,0.200000,16296000,1.308761,42.105261,0.347698
4,1997-10-14 00:00:00-04:00,0.198958,30936000,1.283152,46.601793,0.310476
...,...,...,...,...,...,...
6851,2024-12-31 00:00:00-05:00,219.389999,24819700,1.070065,50.749226,0.086124
6852,2025-01-02 00:00:00-05:00,220.220001,33956600,1.061704,44.291792,0.074616
6853,2025-01-03 00:00:00-05:00,224.190002,27515600,1.059974,44.360545,0.068819
6854,2025-01-06 00:00:00-05:00,227.610001,31849800,1.057722,43.023099,0.066103


In [ ]:
env = TradingEnv(data)